#Fine-Tuning GPT-4o-mini
Copyright 2024 Denis Rothman

**August 2025 update** OpenAI is moving ahead at full speed. The GPT-4o-mini model used in this notebook is not supported anymore.

*Open and now use:*
```
Chapter08/Fine_tuning_GPT_4_1_mini_SQuAd.ipynb
```

[OpenAI fine-tuning documentation](https://beta.openai.com/docs/guides/fine-tuning/)

Check the cost of fine-tuning your dataset on OpenAI before running the notebook.

Run this notebook cell by cell to:

1.Download and prepare the SQuAD dataset
Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset.    
2.Fine-tune a model   
3.Run a fine-tuned model

# Installing the environment


In [ ]:
#You can retrieve your API key from a file(1)
# or enter it manually(2)
#Comment this cell if you want to enter your key manually.
#(1)Retrieve the API Key from a file
#Store you key in a file and read it(you can type it directly in the notebook but it will be visible for somebody next to you)
from google.colab import drive
drive.mount('/content/drive')
f = open("drive/MyDrive/files/api_key.txt", "r")
API_KEY=f.readline()
f.close()

Mounted at /content/drive


In [ ]:
try:
  import openai
except:
  !pip install openai==1.42.0
  import openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


In [ ]:
#(2) Enter your manually by
# replacing API_KEY by your key.
#The OpenAI Key
import os
os.environ['OPENAI_API_KEY'] =API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
!pip install jsonlines==4.0.0

In [ ]:
!pip install datasets==2.20.0

Listing the installed packages

In [ ]:
import subprocess

# Run pip list and capture the output
result = subprocess.run(['pip', 'list'], stdout=subprocess.PIPE, text=True)

# Split the output into lines and count them
package_list = result.stdout.split('\n')

# Adjust count for headers or empty lines
package_count = len([line for line in package_list if line.strip() != '']) - 2

print(f"Number of installed packages: {package_count}")

Number of installed packages: 513


In [ ]:
import subprocess

# Run pip list and capture the output
result = subprocess.run(['pip', 'list'], stdout=subprocess.PIPE, text=True)

# Print the output
print(result.stdout)

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.32.1
aiohappyeyeballs                 2.4.0
aiohttp                          3.10.5
aiosignal                        1.3.1
alabaster                        0.7.16
albucore                         0.0.13
albumentations                   1.4.14
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.18.0
asn1crypto                       1.5.1
astropy                          6.1.2
astropy-iers-data                0.2024.8.26.0.31.57
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            24.2.0
audioread              

counting the number of packages

# 1.Preparing the dataset for fine-tuning

## 1.1.Downloading and displaying the dataset

In [ ]:
from datasets import load_dataset
import pandas as pd

# Load the SQuAD dataset from HuggingFace
dataset = load_dataset("squad", split="train[:500]")

# Filter the dataset to ensure context and answer are present
filtered_dataset = dataset.filter(lambda x: x["context"] != "" and x["answers"]["text"] != [])

# Extract prompt (context + question) and response (answer)
def extract_prompt_response(example):
    return {
        "prompt": example["context"] + " " + example["question"],
        "response": example["answers"]["text"][0]  # Take the first answer
    }

filtered_dataset = filtered_dataset.map(extract_prompt_response)

# Print the number of examples
print("Number of examples: ", len(filtered_dataset))

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Number of examples:  500


In [ ]:
# Convert the filtered dataset to a pandas DataFrame
df_view = pd.DataFrame(filtered_dataset)

# Display the DataFrame
df_view.head()

,id,title,context,question,answers,prompt,response
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ...","Architecturally, the school has a Catholic cha...",Saint Bernadette Soubirous
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe...","Architecturally, the school has a Catholic cha...",a copper statue of Christ
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'...","Architecturally, the school has a Catholic cha...",the Main Building
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...,"Architecturally, the school has a Catholic cha...",a Marian place of prayer and reflection
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...,"Architecturally, the school has a Catholic cha...",a golden statue of the Virgin Mary


## 1.2A Streaming the output to JSON


In [ ]:
import json
import pandas as pd

## 1.2. Preparing the dataset for fine-tuning

In [ ]:
import jsonlines
import pandas as pd
from datasets import load_dataset

# Convert to DataFrame and clean
df = pd.DataFrame(filtered_dataset)
#columns_to_drop = ['title','question','answers']
#df = df.drop(columns=columns_to_drop)

# Prepare the data items for JSON lines file
items = []
for idx, row in df.iterrows():
    detailed_answer = row['response'] + " Explanation: " + row['context']
    items.append({
        "messages": [
            {"role": "system", "content": "Given a SQuAD question built from Wikipedia with crowdworders, provide the correct answer with a detailed explanation."},
            {"role": "user", "content": row['question']},
            {"role": "assistant", "content": detailed_answer}
        ]
    })

# Write to JSON lines file
with jsonlines.open('/content/QA_prompts_and_completions.json', 'w') as writer:
    writer.write_all(items)

### Visualizing the JSON file

In [ ]:
dfile="/content/QA_prompts_and_completions.json"

In [ ]:
import pandas as pd

# Load the data
df = pd.read_json(dfile, lines=True)
df

,messages
0,"[{'role': 'system', 'content': 'Given a SQuAD ..."
1,"[{'role': 'system', 'content': 'Given a SQuAD ..."
2,"[{'role': 'system', 'content': 'Given a SQuAD ..."
3,"[{'role': 'system', 'content': 'Given a SQuAD ..."
4,"[{'role': 'system', 'content': 'Given a SQuAD ..."
...,...
495,"[{'role': 'system', 'content': 'Given a SQuAD ..."
496,"[{'role': 'system', 'content': 'Given a SQuAD ..."
497,"[{'role': 'system', 'content': 'Given a SQuAD ..."
498,"[{'role': 'system', 'content': 'Given a SQuAD ..."


# 2.Fine-tuning the model



In [ ]:
from openai import OpenAI
import jsonlines
client = OpenAI()
# Uploading the training file

result_file = client.files.create(
  file=open("QA_prompts_and_completions.json", "rb"),
  purpose="fine-tune"
)

print(result_file)
param_training_file_name = result_file.id
print(param_training_file_name)

# Creating the fine-tuning job

ft_job = client.fine_tuning.jobs.create(
  training_file=param_training_file_name,
  model="gpt-4o-mini-2024-07-18"
)

# Printing the fine-tuning job
print(ft_job)

FileObject(id='file-1OyEhi0D2b1kcL54JbQ3P1Pa', bytes=645098, created_at=1725287194, filename='QA_prompts_and_completions.json', object='file', purpose='fine-tune', status='processed', status_details=None)
file-1OyEhi0D2b1kcL54JbQ3P1Pa
FineTuningJob(id='ftjob-gQGiuvPMvSop0tzGaDn1NMql', created_at=1725287195, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-h2Kjmcir4wyGtqq1mJALLGIb', result_files=[], seed=1559529989, status='validating_files', trained_tokens=None, training_file='file-1OyEhi0D2b1kcL54JbQ3P1Pa', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)


## Monitoring the fine-tunes

In [ ]:
import pandas as pd
from openai import OpenAI
client = OpenAI()
# Assume client is already set up and authenticated
response = client.fine_tuning.jobs.list(limit=3)# increase to see history

# Initialize lists to store the extracted data
job_ids = []
created_ats = []
statuses = []
models = []
training_files = []
error_messages = []
fine_tuned_models = []  # List to store the fine-tuned model names

# Iterate over the jobs in the response
for job in response.data:
    job_ids.append(job.id)
    created_ats.append(job.created_at)
    statuses.append(job.status)
    models.append(job.model)
    training_files.append(job.training_file)
    error_message = job.error.message if job.error else None
    error_messages.append(error_message)

    # Append the fine-tuned model name
    fine_tuned_model = job.fine_tuned_model if hasattr(job, 'fine_tuned_model') else None
    fine_tuned_models.append(fine_tuned_model)

# Create a DataFrame
df = pd.DataFrame({
    'Job ID': job_ids,
    'Created At': created_ats,
    'Status': statuses,
    'Model': models,
    'Training File': training_files,
    'Error Message': error_messages,
    'Fine-Tuned Model': fine_tuned_models  # Include the fine-tuned model names
})

# Convert timestamps to readable format
df['Created At'] = pd.to_datetime(df['Created At'], unit='s')
df = df.sort_values(by='Created At', ascending=False)

# Display the DataFrame
df

,Job ID,Created At,Status,Model,Training File,Error Message,Fine-Tuned Model
0,ftjob-gQGiuvPMvSop0tzGaDn1NMql,2024-09-02 14:26:35,running,gpt-4o-mini-2024-07-18,file-1OyEhi0D2b1kcL54JbQ3P1Pa,None,None
1,ftjob-oVB0RAcwn3NEi4u0qOMeMZUF,2024-09-02 14:11:28,succeeded,gpt-4o-mini-2024-07-18,file-0dxQmL84uLME7ehnGIjQAxit,None,ft:gpt-4o-mini-2024-07-18:personal::A32VfYIz
2,ftjob-MRnsTGSEAn4Hx7jmDozINtCh,2024-09-02 13:01:03,succeeded,gpt-4o-mini-2024-07-18,file-xF33esEaIDZ2K4JGakTrny6l,None,ft:gpt-4o-mini-2024-07-18:personal::A31WMCRi


### Make sure to obtain your fine-tune model here

If your OpenAI notifications are activated you should receive an email.

Otherwise run the "Monitoring the fine-tunes" cell above to check the status of your fine-tune job.

In [ ]:
import pandas as pd

generation=False  # False until the last model fine-tuned is found. Make sure it used the dataset you trained it on!
# Attempt to find the first non-empty Fine-Tuned Model
non_empty_models = df[df['Fine-Tuned Model'].notna() & (df['Fine-Tuned Model'] != '')]

if not non_empty_models.empty:
    first_non_empty_model = non_empty_models['Fine-Tuned Model'].iloc[0]
    print("The latest fine-tuned model is:", first_non_empty_model)
    generation=True
else:
    first_non_empty_model='None'
    print("No fine-tuned models found.")

The latest fine-tuned model is: ft:gpt-4o-mini-2024-07-18:personal::A32VfYIz


In [ ]:
# Fine-tuned model found(True) or not(False)
generation

True

*Note:* Only continue to Step 3, to use the fine-tuned model when your fine-tuned model is ready. If your OpenAI notifications is activiated, you will receive an email with the status of your fine-tunning job.

# 3.Using the fine-tuned OpenAI model

Note: The is a fine-tuning. As such, be patient!
Rune the `Monitoring the fine-tunes` cell and the f`irst_non_empty_model` cell from time to time.

If the fine-tunning succeeded and your model is ready, the name of your model will be `first_non_empty_model`

1.Go to the OpenAI Playground to test your model: https://platform.openai.com/playground

2.Check the metrics in the fine-tuning UI:
https://platform.openai.com/finetune/

3.Try the fined-tune model out in the cell below.

In [ ]:
# Define the prompt
prompt="Which prize did Frederick Buechner create?"

*Note:* Only run the following cell if your fine-tune job has succeeded and a fined-tuned model is found in the *Monitoring the fine-tunes"* section of *2.Fine-tuning the model.*

In [ ]:
# Assume first_non_empty_model is defined above this snippet
if generation==True:
    response = client.chat.completions.create(
        model=first_non_empty_model,
        temperature=0.0,  # Adjust as needed for variability
        messages=[
            {"role": "system", "content": "Given a question, reply with a complete explanation for students."},
            {"role": "user", "content": prompt}
        ]
    )
else:
    print("Error: Model is None, cannot proceed with the API request.")

In [ ]:
if generation==True:
  print(response)

ChatCompletion(id='chatcmpl-A32gXi3odDcuI5Ad4p1l6VK4T9fbd', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The Frederick Buechner Prize for Preaching Explanation: The Rev. Dr. Frederick Buechner served as Writer in Residence at Princeton Theological Seminary from 1988 to 1989. During this time he wrote The Book of Buechner, a collection of excerpts from his works on the themes of faith and religion, with a commentary by the Rev. Dr. Thomas R. Steagald. Buechner's novels include The Storm, The Entrance to Porlock, and Godric, which was a finalist for the National Book Award. His work of nonfiction, Telling the Truth: The Gospel as Comedy, Tragedy, and Fairy Tale, argues that the gospel can be understood in terms of three genres of literature: comedy, tragedy, and fairy tale.", refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1725288669, model='ft:gpt-4o-mini-2024-07-18:personal::A32VfYIz', object='chat.

In [ ]:
if (generation==True):
  # Access the response from the first choice
  response_text = response.choices[0].message.content
  # Print the response
  print(response_text)

The Frederick Buechner Prize for Preaching Explanation: The Rev. Dr. Frederick Buechner served as Writer in Residence at Princeton Theological Seminary from 1988 to 1989. During this time he wrote The Book of Buechner, a collection of excerpts from his works on the themes of faith and religion, with a commentary by the Rev. Dr. Thomas R. Steagald. Buechner's novels include The Storm, The Entrance to Porlock, and Godric, which was a finalist for the National Book Award. His work of nonfiction, Telling the Truth: The Gospel as Comedy, Tragedy, and Fairy Tale, argues that the gospel can be understood in terms of three genres of literature: comedy, tragedy, and fairy tale.


In [ ]:
import textwrap

if generation==True:
  wrapped_text = textwrap.fill(response_text.strip(), 60)
  print(wrapped_text)

The Frederick Buechner Prize for Preaching Explanation: The
Rev. Dr. Frederick Buechner served as Writer in Residence at
Princeton Theological Seminary from 1988 to 1989. During
this time he wrote The Book of Buechner, a collection of
excerpts from his works on the themes of faith and religion,
with a commentary by the Rev. Dr. Thomas R. Steagald.
Buechner's novels include The Storm, The Entrance to
Porlock, and Godric, which was a finalist for the National
Book Award. His work of nonfiction, Telling the Truth: The
Gospel as Comedy, Tragedy, and Fairy Tale, argues that the
gospel can be understood in terms of three genres of
literature: comedy, tragedy, and fairy tale.


[Consult OpenAI fine-tune documentation for more](https://platform.openai.com/docs/guides/fine-tuning)